# Mount my Google Drive where I stored the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Grab YOLOv7 model from git repo



In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

# Installing requirements

In [ ]:
!cd yolov7 && pip install -r requirements.txt

# Download weights of pretrained model - want yolov7_training.pt for weights of model configured to work well for fine-tuning

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
!mv yolov7.pt ./yolov7/yolov7.pt

# Download config files from my github repo

In [ ]:
!rm -rf ./nzbird_detector
!git clone https://github.com/cravies/nzbird_detector

# Move training, testing and validation images and labels to correct place

In [ ]:
!cp ./drive/MyDrive/bird_data_2/trains.zip ./trains.zip

In [ ]:
!unzip ./trains.zip

In [ ]:
!rm -rf ./yolov7/train
!mkdir ./yolov7/train

!rm -rf ./yolov7/test
!mkdir ./yolov7/test

!rm -rf ./yolov7/val
!mkdir ./yolov7/val

!mkdir ./yolov7/train/images
!mkdir ./yolov7/train/labels

!mkdir ./yolov7/test/images
!mkdir ./yolov7/test/labels

!mkdir ./yolov7/val/images
!mkdir ./yolov7/val/labels

!cp -r ./trains/train/* ./yolov7/train/images
!cp -r ./trains/train_out/* ./yolov7/train/labels

!cp -r ./drive/MyDrive/bird_data_2/val/* ./yolov7/val/images
!cp -r ./drive/MyDrive/bird_data_2/val_out/* ./yolov7/val/labels

!cp -r ./drive/MyDrive/bird_data_2/test/vanilla/* ./yolov7/test/images
!cp -r ./drive/MyDrive/bird_data_2/test_out/* ./yolov7/test/labels

# Double check for test train val leakage

In [ ]:
import os
import collections

def dupes():
    """
    Just double checking if train test val have any overlapping files
    """
    folds = ["./yolov7/test/images","./yolov7/val/images","./yolov7/train/images"]
    files=[]
    for folder in folds:
        files.append(os.listdir(folder))
    res = []
    for folder in files:
        for file in folder:
            res.append(file)
    #check if any files appear twice
    print("duplicates: ",
        [item for item,count in collections.Counter(res).items() if count > 1]
    )

dupes()

# Grab custom.yaml file from git repo to define my dataset format, classes etc

In [ ]:
!mv ./nzbird_detector/custom.yaml ./yolov7/data/custom.yaml

# Grab our 100 epochs trained model saved on google drive

In [ ]:
!mkdir ./yolov7/runs/
!mkdir ./yolov7/runs/train
!rm -rf ./yolov7/runs/train/final
!mkdir ./yolov7/runs/train/final
!cp -r ./drive/MyDrive/long_run/* ./yolov7/runs/train/final

# Finetune our pretrained model on our train images for 100 epochs

 # Use tensorboard for monitoring training

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./yolov7/runs/train

In [ ]:
!cd ./yolov7 && python3 train.py --weights ./yolov7.pt --data "data/custom.yaml" --workers 8 --device 0 --batch-size 4 --img 400 --cfg cfg/training/yolov7.yaml --hyp data/hyp.scratch.p5.yaml --epochs 100

# Run object detection on image test set
need to specify weights with --weights based on where they are saved

In [ ]:
!cd yolov7 && python detect.py --weights runs/train/final/weights/best.pt --conf-thres 0.07 --iou-thres 0.07 --source "./test/images/"

# Copy the results so I can look at them later

In [ ]:
!cp -r ./yolov7/runs/detect/exp4 ./drive/MyDrive/

# Download videos of birds to run inference on

In [ ]:
!pip install pytube

In [ ]:
from pytube import YouTube
#tui
YouTube('https://m.youtube.com/watch?v=ucvO-gphZng').streams[1].download()
#takahe
YouTube('https://m.youtube.com/watch?v=XRC4VATT9i8').streams[1].download()
#kea 
YouTube('https://m.youtube.com/watch?v=OXqhYAwSask').streams[1].download()

# rename them (must do this manually)

# Now run inference on bird videos (have to tweak confidence threshold and iou threshold manually to work)

In [ ]:
!cd yolov7 && python detect.py --weights runs/train/final/weights/best.pt --source "../kea.mp4" --conf-thres 0.45 --iou-thres 0.62

# Copy the results back to my drive so I can watch them later

In [ ]:
cp ./yolov7/runs/detect/exp10/kea.mp4 ./drive/MyDrive/kea_final.mp4